## Diego Orejuela
## Winter 2022 Data Science Intern Challenge

In [1]:
# import basic libraries
import pandas as pd
import numpy as np
import xlsxwriter
from datetime import datetime as dt
from datetime import timedelta

# Question 1

In [2]:
# load dataset to a dataframe
df = pd.read_excel('2019 Winter Data Science Intern Challenge Data Set.xlsx')
df.tail()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
4995,4996,73,993,330,2,debit,2017-03-30 13:47:16.597
4996,4997,48,789,234,2,cash,2017-03-16 20:36:16.389
4997,4998,56,867,351,3,cash,2017-03-19 05:42:42.228
4998,4999,60,825,354,2,credit_card,2017-03-16 14:51:18.188
4999,5000,44,734,288,2,debit,2017-03-18 15:48:18.205


In [3]:
# become familiar with the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        5000 non-null   int64         
 1   shop_id         5000 non-null   int64         
 2   user_id         5000 non-null   int64         
 3   order_amount    5000 non-null   int64         
 4   total_items     5000 non-null   int64         
 5   payment_method  5000 non-null   object        
 6   created_at      5000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 273.6+ KB


## Since we want to look at a 30 day window. Use datetime accessors to make sure dates are within the window

In [4]:
df_1 = df.copy()
df_1['year'] = df_1['created_at'].dt.year
df_1['month'] = df_1['created_at'].dt.month
# df_1['day'] = df_1['created_at'].dt.day
df_1['YYYYMMDD'] = df_1['created_at'].dt.strftime('%Y-%m-%d').values.astype('datetime64')
df_1.tail()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at,year,month,day,YYYYMMDD
4995,4996,73,993,330,2,debit,2017-03-30 13:47:16.597,2017,3,30,2017-03-30
4996,4997,48,789,234,2,cash,2017-03-16 20:36:16.389,2017,3,16,2017-03-16
4997,4998,56,867,351,3,cash,2017-03-19 05:42:42.228,2017,3,19,2017-03-19
4998,4999,60,825,354,2,credit_card,2017-03-16 14:51:18.188,2017,3,16,2017-03-16
4999,5000,44,734,288,2,debit,2017-03-18 15:48:18.205,2017,3,18,2017-03-18


In [9]:
# make sure year is consistent
df_1['year'].value_counts()

2017    5000
Name: year, dtype: int64

In [10]:
# make sure month is consistent
df_1['month'].value_counts()

3    5000
Name: month, dtype: int64

## After becoming familiar with the dataset and making sure the dates are not contributing to the calculation mistake. The next step is to perform the AOV calculation:
## Average Order Value = Revenue / Number of Orders
## Now it's time to calculate the AOV with the columns: order_amount and total_items.
## Metrics:
## The sum of the column order_amount will be the numerator
## The sum of the column total_items will be the denominator

In [16]:
# Use the sum function over the selected columns and divide

order_amt = df_1['order_amount'].sum()
items_tot = df_1['total_items'].sum()
aov = order_amt / items_tot
aov

357.92152221412965

In [17]:
aov_price = "${:,.2f}".format(aov)
print('Shopify Sneaker Shops AOV: ' + aov_price)

Shopify Sneaker Shops AOV: $357.92


## Since the denominator for the AOV is Number of orders, a common mistake in the calculations would be to Count the number of orders. 
## In this case, perform a count of the number of total_items, but there can be more than one item per order. Therefore, we need to sum the number of items per order.

In [18]:
order_amt = df_1['order_amount'].sum()
items_tot = df_1['total_items'].count()
aov_wrong = order_amt / items_tot
aov_price_wrong = "${:,.2f}".format(aov_wrong)
print('Shopify Sneaker Shops AOV is not : ' + aov_price_wrong)

Shopify Sneaker Shops AOV is not : $3,145.13


# Question 2

In [5]:
# a. How many orders were shipped by Speedy Express in Total?

'''
SELECT s.ShipmentName, COUNT(o.OrderID) as OrderCount
FROM Shippers s, Orders o
WHERE s.ShipperID = o.ShipperID
AND s.ShipperName = 'Speedy Express'
'''

# Answer:

print('ShipperName: Speedy Express')
print('OrderCount: 54')

ShipperName: Speedy Express
OrderCount: 54


In [6]:
# b. What is the last name with the employee with the most orders?

'''
SELECT count(o.OrderID) as OrderCount, e.LastName
FROM Orders o, Employees e
WHERE o.EmployeeID = e.EmployeeID
GROUP BY e.EmployeeID
ORDER BY OrderCount DESC
LIMIT 1
'''

#Alternate Solution
'''
SELECT max(t1.OrderCount) as OrderCount, e.LastName
FROM Employees e,
(
    select count(o.OrderID) as OrderCount, o.EmployeeID
    from Orders o
    group by o.EmployeeID
) t1
WHERE t1.EmployeeID = e.EmployeeID
'''

# Answer:
print('LastName: Peacock')
print('OrderCount: 40')

LastName: Peacock
OrderCount: 40


In [7]:
# c. What product was ordered the most by customers in Germany?

'''
with Base as (
SELECT p.ProductName , o.OrderID, od.Quantity, c.Country
FROM Customers c, Products p, Orders o, OrderDetails od 
WHERE c.Country = 'Germany'
AND od.ProductID = p.ProductID
AND o.CustomerID = c.CustomerID)

SELECT ProductCounts.*, (ProductCount*Quantity) as OrdersTotal
FROM
(
    SELECT ProductName, count(ProductName) as ProductCount, Quantity
    FROM Base
    GROUP BY ProductName
) ProductCounts

ORDER BY OrdersTotal DESC
LIMIT 1

'''
# Answer: 
print('ProductName: Camembert Pierrot')
print('ProductCount: 300')
print('Quantity: 40')
print('OrdersTotal: 12000')

ProductName: Camembert Pierrot
ProductCount: 300
Quantity: 40
OrdersTotal: 12000
